In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [6]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

### 0. Data processing  
    - histogram에 대한 처리: 대표값으로 대체
        - 평균, 분산
    - NA 값에 대한 처리: columnwise 적용. pos null 30%이상 -> pos median, 나머지 -> 전체의 median 
    - class imbalance: oversampling (SMOTE)
    - feature selection: drop over 40% na, 2장 내용 참고, logistic 회귀
    - feature reduction: PCA
    - columnwise normalizing

#### 1) dtype processing

In [7]:
data = pd.read_csv("./Train_data.csv", index_col=0)

# class to numeric; pos: 1, neg: 0
data['class'] = data['class'].replace('neg', 0)
data['class'] = data['class'].replace('pos', 1)

# na, numeric processing
data = data.replace('na', np.nan) # change 'na' ro np.nan
data = data.apply(pd.to_numeric) # change object to int or float

#### 2) Feature Selection
- Drop if null ratio over 0,4; gap btw 25~40%

#### 3) NA's: replace depending on column characteristic.
- If pos_null_ratio>=0.3, pos median. Else total median; If pos_null_ratio<0.3, many cols also have high neg_null_ratio
    

In [8]:
# check null columns per cats
pos_data = data[data["class"]==1] 
neg_data = data[data["class"]==0] 

pos_df, neg_df = pos_data.isnull().sum().to_frame(name="pos_null_cnt"), neg_data.isnull().sum().to_frame(name="neg_null_cnt")
null_check = pos_df.join(neg_df)
null_check["total_null"] = null_check["pos_null_cnt"] + null_check["neg_null_cnt"]
null_check.insert(1, "pos_cnt", len(pos_data))
null_check.insert(3, "neg_cnt", len(neg_data))

null_check["pos_null_ratio"] = null_check.pos_null_cnt/null_check.pos_cnt
null_check["neg_null_ratio"] = null_check.neg_null_cnt/null_check.neg_cnt
null_check["total_null_ratio"] = (null_check.pos_null_cnt + null_check.neg_null_cnt)/len(data)

# drop cols w/ too many na's
to_drop = null_check[null_check.total_null_ratio>=0.4]
data.drop(to_drop.index, inplace=True, axis=1)

# na replacement
POS_MEDIAN = pos_data.median(axis=0).to_dict()
TOTAL_MEDIAN = data.median(axis=0).to_dict()

with_pos = data.loc[:, null_check.pos_null_ratio>=0.4]
with_pos = with_pos.index
data.fillna(value=POS_MEDIAN, inplace=True)

with_total = data.loc[:, null_check.pos_null_ratio<0.4]
with_total = with_total.index
data.fillna(value=TOTAL_MEDIAN, inplace=True)

# data.isnull().sum().sum()

#### 4)Standardize_ Rowise
`minmax scaling과 성능 차이 시도 가능`

In [9]:
X = data.iloc[:, 1:]
y = data["class"]

scaler = StandardScaler()
scaler.fit_transform(X)
X_scaled = scaler.fit_transform(X) # standardizing if needed
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
X_scaled.describe().applymap("{0:.2f}".format)

aa_000    ac_000    ad_000    ae_000    af_000    ag_000    ag_001  \
count  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00   
mean       0.00      0.00     -0.00      0.00     -0.00      0.00     -0.00   
std        1.00      1.00      1.00      1.00      1.00      1.00      1.00   
min       -0.26     -0.43     -0.00     -0.04     -0.05     -0.01     -0.02   
25%       -0.26     -0.43     -0.00     -0.04     -0.05     -0.01     -0.02   
50%       -0.13     -0.43     -0.00     -0.04     -0.05     -0.01     -0.02   
75%       -0.05     -0.43     -0.00     -0.04     -0.05     -0.01     -0.02   
max      183.89      2.31    238.74    127.76     97.00    160.61    181.76   

         ag_002    ag_003    ag_004    ag_005    ag_006    ag_007    ag_008  \
count  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00   
mean       0.00     -0.00      0.00      0.00     -0.00      0.00     -0.00   
std        1.00      1.00      1.00      1.00      1.00      1.00      1.00   
min       -0.06     -0.12     -0.19     -0.35     -0.43     -0.35     -0.16   
25%       -0.06     -0.12     -0.19     -0.35     -0.43     -0.35     -0.16   
50%       -0.06     -0.12     -0.19     -0.30     -0.19     -0.27     -0.15   
75%       -0.06     -0.12     -0.17     -0.08      0.05      0.07     -0.03   
max      102.34     83.07     87.71     50.27     23.06     43.54     75.47   

         ag_009    ah_000    ai_000    aj_000    ak_000    al_000      am_0  \
count  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00   
mean       0.00     -0.00      0.00      0.00     -0.00     -0.00     -0.00   
std        1.00      1.00      1.00      1.00      1.00      1.00      1.00   
min       -0.03     -0.44     -0.05     -0.02     -0.01     -0.12     -0.12   
25%       -0.03     -0.43     -0.05     -0.02     -0.01     -0.12     -0.12   
50%       -0.03     -0.22     -0.05     -0.02     -0.01     -0.12     -0.12   
75%       -0.03     -0.08     -0.05     -0.02     -0.01     -0.12     -0.12   
max      131.63     14.78     95.71     97.66    125.05     62.68     58.90   

         an_000    ao_000    ap_000    aq_000    ar_000    as_000    at_000  \
count  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00   
mean      -0.00      0.00     -0.00      0.00     -0.00      0.00     -0.00   
std        1.00      1.00      1.00      1.00      1.00      1.00      1.00   
min       -0.45     -0.45     -0.34     -0.36     -0.09     -0.01     -0.04   
25%       -0.44     -0.44     -0.33     -0.36     -0.09     -0.01     -0.04   
50%       -0.22     -0.22     -0.23     -0.23     -0.09     -0.01     -0.04   
75%       -0.07     -0.08     -0.12     -0.08     -0.09     -0.01     -0.04   
max       16.50     18.77     34.05     20.91     64.63    196.71     88.57   

         au_000    av_000    ax_000    ay_000    ay_001    ay_002    ay_003  \
count  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00   
mean       0.00      0.00      0.00     -0.00      0.00     -0.00      0.00   
std        1.00      1.00      1.00      1.00      1.00      1.00      1.00   
min       -0.01     -0.18     -0.27     -0.02     -0.02     -0.03     -0.04   
25%       -0.01     -0.18     -0.26     -0.02     -0.02     -0.03     -0.04   
50%       -0.01     -0.16     -0.22     -0.02     -0.02     -0.03     -0.04   
75%       -0.01     -0.06     -0.05     -0.02     -0.02     -0.03     -0.04   
max      195.44    117.69     79.95    113.17    132.24     69.86     61.74   

         ay_004    ay_005    ay_006    ay_007    ay_008    ay_009    az_000  \
count  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00  57000.00   
mean      -0.00      0.00     -0.00      0.00     -0.00      0.00      0.00   
std        1.00      1.00      1.00      1.00      1.00      1.00      1.00   
min       -0.03     -0.08     -0.32     -0.31     -0.27     -0.01     -0.11   
25%       -0.03     -0.08     -0.32     -0.31     -0.2

#### 5) Class imbalance: oversampling using SMOTE

#### 6) Feature dim. reduction: PCA
`feature weight 확인 원한다면 해제 필요`

`return 값: X, y`